<a href="https://colab.research.google.com/github/fbeilstein/topological_data_analysis/blob/master/particle_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Dependencies
!apt-get update
!apt-get install -y cmake g++ make libboost-dev libboost-system-dev libboost-filesystem-dev libboost-thread-dev
!apt-get install -y libeigen3-dev

# Install OpenMPI
!apt-get install -y libopenmpi-dev openmpi-bin

# Install other dependencies
!pip install cython numpy mpi4py pybind11

# Install cereal
!git clone https://github.com/USCiLab/cereal.git
!mkdir -p /usr/local/include/cereal
!cp -r cereal/include/cereal /usr/local/include/

# Clone the HOOMD-blue repository
!git clone --recursive https://github.com/glotzerlab/hoomd-blue.git
%cd hoomd-blue

# Build HOOMD-blue from Source
!mkdir build
%cd build

# Get pybind11 include path
import pybind11
pybind11_include = pybind11.get_include()
pybind11_cmake = pybind11.get_cmake_dir()
print("CMAKE: ", pybind11_cmake)

# Run cmake with pybind11 include directory
!cmake ../ -DENABLE_MPI=ON -DENABLE_GPU=ON -DPYTHON_EXECUTABLE=/usr/bin/python3 -Dpybind11_DIR={pybind11_cmake}
!make -j4
!make install

In [9]:
import os
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/lib'
os.environ['HOOMD_PATH'] = '/content/hoomd-blue/build'

In [16]:
import hoomd
import hoomd.md

# Initialize HOOMD-blue context
hoomd.context.initialize("--mode=gpu")

# Create a simulation box with particles
N = 1000
L = 10.0
snapshot = hoomd.data.make_snapshot(N=N, box=hoomd.data.boxdim(L=L), particle_types=['A'])

import numpy as np
snapshot.particles.position[:] = np.random.uniform(-L/2, L/2, size=(N, 3))

system = hoomd.init.read_snapshot(snapshot)

# Custom potential function
def custom_force(r, rmin, rmax, epsilon, sigma):
    # Your custom force calculation here
    force = -4 * epsilon * ((12 * (sigma ** 12) / (r ** 13)) - (6 * (sigma ** 6) / (r ** 7)))
    return force

# Custom energy function
def custom_energy(r, rmin, rmax, epsilon, sigma):
    # Your custom energy calculation here
    energy = 4 * epsilon * ((sigma / r) ** 12 - (sigma / r) ** 6)
    return energy

# Define the custom potential
nl = hoomd.md.nlist.cell()
user_potential = hoomd.md.pair.user(r_cut=2.5, nlist=nl)
user_potential.set_params(func=custom_force, rmin=0.1, rmax=2.5, coeff=dict(epsilon=1.0, sigma=1.0))

# Integrate the equations of motion
hoomd.md.integrate.mode_standard(dt=0.005)
integrator = hoomd.md.integrate.nve(group=hoomd.group.all())

# Run the simulation
hoomd.run(1e4)


AttributeError: module 'hoomd.data' has no attribute 'make_snapshot'

In [ ]:
!zip -r hoomd_compiled.zip /usr/local/lib/python3.10/dist-packages/hoomd

In [ ]:
!unzip hoomd_compiled.zip -d /

https://hoomd-blue.readthedocs.io/en/v4.8.2/